In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("bank-full.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [9]:
print("Jumlah missing value per kolom:")
print(df.isnull().sum())

Jumlah missing value per kolom:
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


In [10]:
print("Jumlah nilai 'unknown' per kolom kategorikal:")
for col in df.select_dtypes(include='object'):
    print(f"{col}: {(df[col] == 'unknown').sum()}")

# Ganti nilai unknown dengan modus (kategori paling sering)
for col in df.select_dtypes(include='object'):
    df[col] = df[col].replace('unknown', df[col].mode()[0])

Jumlah nilai 'unknown' per kolom kategorikal:
job: 0
marital: 0
education: 0
default: 0
housing: 0
loan: 0
contact: 0
month: 0
poutcome: 28069
y: 0


In [11]:
print("Jumlah duplikat sebelum dibersihkan:", df.duplicated().sum())
df = df.drop_duplicates()
print("Jumlah data setelah menghapus duplikat:", df.shape[0])

Jumlah duplikat sebelum dibersihkan: 0
Jumlah data setelah menghapus duplikat: 28069


In [13]:
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)]

print("Jumlah data setelah menghapus outlier:", df.shape[0])

Jumlah data setelah menghapus outlier: 23943


In [14]:
scaler = StandardScaler()
num_cols = df.select_dtypes(include=np.number).columns
df[num_cols] = scaler.fit_transform(df[num_cols])
df[num_cols].head()

,age,balance,day,duration,campaign,pdays,previous
1,0.392501,-0.704453,-1.27887,-0.310905,-0.888999,0.0,0.0
2,-0.729497,-0.757571,-1.27887,-0.962324,-0.888999,0.0,0.0
3,0.698500,2.201335,-1.27887,-0.823354,-0.888999,0.0,0.0
4,-0.729497,-0.759539,-1.27887,0.097318,-0.888999,0.0,0.0
5,-0.525497,-0.307046,-1.27887,-0.415132,-0.888999,0.0,0.0


In [15]:
cat_cols = df.select_dtypes(include='object').columns
encoder = LabelEncoder()

for col in cat_cols:
    df[col] = encoder.fit_transform(df[col])

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
1,0.392501,9,2,1,0,-0.704453,1,0,0,-1.27887,8,-0.310905,-0.888999,0.0,0.0,0,0
2,-0.729497,2,1,1,0,-0.757571,1,1,0,-1.27887,8,-0.962324,-0.888999,0.0,0.0,0,0
3,0.698500,1,1,1,0,2.201335,1,0,0,-1.27887,8,-0.823354,-0.888999,0.0,0.0,0,0
4,-0.729497,1,2,1,0,-0.759539,0,0,0,-1.27887,8,0.097318,-0.888999,0.0,0.0,0,0
5,-0.525497,4,1,2,0,-0.307046,1,0,0,-1.27887,8,-0.415132,-0.888999,0.0,0.0,0,0


In [16]:
df['balance_per_day'] = df['balance'] / (df['day'] + 1)
df['balance_per_day'].head()

1    2.526098
2    2.716576
3   -7.893770
4    2.723631
5    1.101038
Name: balance_per_day, dtype: float64

In [17]:
X = df.drop('y', axis=1)
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Data training:", X_train.shape)
print("Data testing:", X_test.shape)

Data training: (19154, 17)
Data testing: (4789, 17)


1. Data Cleaning
Tahap ini bertujuan untuk memastikan bahwa data yang digunakan sudah bersih dan layak untuk analisis.

Pengecekan Missing Value:
Setelah diperiksa menggunakan fungsi isnull().sum(), tidak ditemukan nilai kosong pada dataset.

Penanganan Nilai “unknown”:
Beberapa kolom kategorikal seperti job, education, dan poutcome memiliki nilai 'unknown', yang berarti informasi tidak tersedia. Nilai ini diganti dengan modus (kategori yang paling sering muncul) agar kolom tersebut tetap konsisten dan tidak mengganggu hasil analisis.

Pengecekan Data Duplikat:
Pemeriksaan dengan duplicated() menunjukkan tidak ada data yang terduplikasi.

Penanganan Outlier:
Outlier dideteksi pada kolom numerik dengan metode IQR (Interquartile Range). Data yang nilainya terlalu ekstrem di luar batas wajar dihapus.

Kesimpulan: Setelah proses cleaning, dataset menjadi lebih rapi, bebas dari nilai yang hilang, kategori tidak jelas, duplikat, dan nilai ekstrem.

2. Normalisasi / Standarisasi Kolom Numerik
Beberapa kolom seperti age, balance, duration, dan campaign memiliki rentang nilai yang sangat berbeda.

Metode yang digunakan adalah StandardScaler dari Scikit-Learn. Setiap kolom numerik diubah sehingga memiliki mean = 0 dan standard deviation = 1.

Alasan:
Normalisasi penting agar algoritma pembelajaran mesin tidak berat sebelah terhadap fitur dengan nilai besar. Misalnya, kolom balance (ribuan) dan campaign (satuan) akan punya pengaruh yang seimbang setelah distandarisasi.

3. Encoding Kolom Kategorikal
Kolom dengan tipe data kategorikal tidak bisa langsung digunakan oleh model karena berisi teks. Maka dilakukan encoding untuk mengubah data menjadi bentuk numerik.

Metode yang digunakan adalah LabelEncoder. Setiap nilai kategori diganti dengan angka unik. Misalnya, kolom job yang berisi “blue-collar”, “management”, “technician”, dll., diubah menjadi angka seperti 0, 1, 2, dan seterusnya.

Alasan:
Encoding diperlukan agar model machine learning bisa membaca dan memproses kolom kategorikal sebagai fitur numerik. Tanpa encoding, algoritma seperti regresi logistik atau decision tree tidak akan bisa berfungsi.

4. Feature Engineering
Tahap ini menambahkan fitur baru agar model mendapatkan informasi tambahan yang mungkin relevan.

Fitur baru yang dibuat adalah balance_per_day yaitu hasil pembagian antara saldo (balance) dan jumlah hari (day) dari kontak terakhir. Fitur ini menggambarkan rata-rata saldo per hari, yang dapat memberikan indikasi perilaku keuangan nasabah.

Alasan:
Feature engineering membantu model mengenali pola baru dari kombinasi fitur yang sudah ada, sehingga dapat meningkatkan akurasi dan pemahaman terhadap data.

5. Splitting Data (Training & Testing)
Tahap terakhir adalah membagi data menjadi dua bagian:
80% untuk data training
20% untuk data testing

Pembagian ini dilakukan menggunakan fungsi train_test_split dari Scikit-Learn.